# Section 1: Importing the libraries and reading the dataset

In [7]:
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
train_path = 'data/train-1.csv'
test_path = 'data/val-1.csv'

In [9]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Section 2: Preparing the dataset